In [ ]:
from typing import NamedTuple

class Airport(NamedTuple):
    airport_id: str
    full_name: str
    country_name: str

In [ ]:
from os import getenv
from connect_db import connect_db
con = connect_db(
    username=getenv("ORACLE_USERNAME"),
    password=getenv("ORACLE_PASSWORD"),
    host=getenv("ORACLE_HOST"),
    port=int(getenv("ORACLE_PORT"))
)
cur = con.cursor()

In [ ]:
from typing import Iterable
import requests
from bs4 import BeautifulSoup, Tag
from functools import cache

@cache
def get_airport_tables() -> Iterable[Tag]:
    # stubborn :v
    HEADERS = {
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36"
    }
    
    ret = requests.get("https://www.nationsonline.org/oneworld/IATA_Codes/airport_code_list.htm", headers=HEADERS)
    ret.raise_for_status()
    
    soup = BeautifulSoup(ret.content, 'lxml')
    yield from soup.find_all("table")
    
    
def get_airport() -> Iterable[Airport]:
    for table in get_airport_tables():
        for row in table.find_all("tr"):
            cols = row.find_all("td")
            if len(cols) < 3:
                continue
                
            airport_id = cols[2].get_text(strip=True)
            airport_name = cols[0].get_text(strip=True)
            country_name = cols[1].get_text(strip=True)
            yield Airport(
                airport_id=airport_id,
                full_name=airport_name,
                country_name=country_name
            )
    
trans = str.maketrans({"'": "''", "&": r"\&"})
make_sql_nice = lambda s: s.translate(trans)

for airport in get_airport():
    country_id = ""
    country_name = airport.country_name
    for row in cur.execute(f"SELECT COUNTRY_ID FROM COUNTRY WHERE CONVERT(COUNTRY_NAME, 'US7ASCII') = CONVERT('{make_sql_nice(country_name)}', 'US7ASCII')"):
        country_id = row[0]
        
    if not country_id:
        if country_name.startswith("USA"):
            country_id = "US"
        if country_name == "United Kingdom":
            country_id = "GB"
        elif country_name.startswith("Croatia"):
            country_id = "HR"
        elif country_name == "Iran":
            country_id = "IR"
        elif country_name == "Turkey":
            country_id = "TR"
        elif country_name == "Channel Islands":
            country_id = "GB"
        elif country_name.endswith("China"):
            country_id = "CN"
        elif country_name.startswith("Comoros"):
            country_id = "KM"
        elif country_name.endswith("Bahamas"):
            country_id = "BS"
        elif country_name.startswith("Switzerland"):
            country_id = "CH"
        elif country_name.startswith("Netherlands"):
            country_id = "NL"
        elif country_name == "Congo (ROC)":
            country_id = "CG"
        elif country_name == "Congo (DRC)":
            country_id = "CD"
        elif country_name.startswith("Saint Vincent"):
            country_id = "VC"
        elif country_name == "Saudi Arabien":
            country_id = "SA"
        elif country_name.startswith("Timor Leste"):
            country_id = "TL"
        elif country_name == "Scotland,UK":
            country_id = "GB"
        elif country_name == "Vietnam":
            country_id = "VN"
        elif country_name == "Russia":
            country_id = "RU"
        elif country_name.startswith("Wallis and Futuna"):
            country_id = "WF"
        elif country_name.startswith("Palestin"):
            country_id = "PS"
        elif country_name.endswith("Spain"):
            country_id = "ES"
        elif country_name in ("Korea South", "South Korea"):
            country_id = "KR"
        elif country_name.endswith("King Island (Australia)"):
            country_id = "AU"
        elif country_name.endswith("Loyaute, Pazifik"):
            country_id = "FR"
        elif country_name.endswith("Norway"):
            country_id = "NO"
        elif country_name.endswith("Syria"):
            country_id = "SY"
        elif country_name == "Tanzania":
            country_id = "TZ"
        elif country_name == "Brunei":
            country_id = "BN"
        elif country_name == "Venezuela":
            country_id = "VE"
        elif country_name == "Moldava":
            country_id = "MD"
        elif country_name == "Bolivia":
            country_id = "BO"
        elif country_name == "Moldova":
            country_id = "MD"
        elif country_name == "Lao PDR":
            country_id = "LA"
        elif country_name == "Taiwan":
            country_id = "TW"
        elif country_name.startswith("Macau"):
            country_id = "MO"
        elif country_name == "Maledives":  # lol wtf
            country_id = "MV"
        elif country_name == "Swaziland":
            country_id = "SZ"
        elif country_name.startswith("Fiji"):
            country_id = "FJ"
        elif country_name == "St. Kitts and Nevis":
            country_id = "KN"
        elif country_name == "Macedonia":
            country_id = "MK"
        elif country_name.startswith("Gabon"):
            country_id = "GA"
        elif country_name.startswith("French Polynesia"):
            country_id = "PF"
        elif country_name.startswith("Micronesia"):
            country_id = "FM"
        elif country_name == "Czech Republic":
            country_id = "CZ"
        elif country_name == "US Minor Outlying Islands":
            country_id = "UM"
        elif country_name == "Cape Verde":
            country_id = "CV"
        elif country_name.startswith("India"):
            country_id = "IN"
        elif country_name == "Cook Island":
            country_id = "CK"
        elif country_name.startswith("Sao Tome"):
            country_id = "ST"
        elif country_name.endswith("Japan"):
            country_id = "JP"
        elif country_name.startswith("St. Martin"):
            country_id = "MF"
        elif country_name == "North Korea":
            country_id = "KP"
        elif country_name == "British Virgin Islands":
            country_id = "VG"
    if country_id:
        print(f"    INTO AIRPORT (AIRPORT_ID, AIRPORT_FULLNAME, COUNTRY_ID) "
              f"VALUES ('{airport.airport_id}', '{make_sql_nice(airport.full_name)}', '{country_id}')")

In [ ]:
con.close()